In [1]:
from pathlib import Path
import torch
import imageio
import numpy
import gymnasium as gym
from lerobot.policies.diffusion.modeling_diffusion import DiffusionPolicy
from lerobot.datasets.lerobot_dataset import LeRobotDataset
from cmirobot import envs
from cmirobot.teleoperators import GamepadTeleop,KeyboardTeleop,KeyboardEndEffectorTeleop

c:\Users\wangzhian\.conda\envs\nvidia\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
import mujoco
import mujoco.viewer
import numpy as np
from pathlib import Path
import glfw

xml_path = Path( "../data/scene/test.xml")
model_path=str(xml_path.resolve())
print(model_path)
model=mujoco.MjModel.from_xml_path(model_path)
data=mujoco.MjData(model)

print("njnt",model.njnt)

print("nbody",model.nbody)
print("xpos",data.xpos)
print("nq",model.nq)
print("qpos",data.qpos)
print(type(data.qpos))
# print(model.jnt_type)
print("nv",model.nv)
print("qvel",data.qvel)
print("data.ctrl",data.ctrl)

print("data.body(base).xpos",data.body("base").xpos)
print("data.body(base).xmat",data.body("base").xmat)
print("data.joint(base_joint).qpos",data.joint("base_joint").qpos)
[mujoco.mj_id2name(model,mujoco.mjtObj.mjOBJ_JOINT,joint_idx)
                                 for joint_idx in range(model.njnt)]


# 设置分辨率
resolution = (800, 600)
# 初始化 OpenGL 上下文（离屏渲染）
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE)
window = glfw.create_window(resolution[0], resolution[1], "Offscreen", None, None)
glfw.make_context_current(window)

print("ncam",model.ncam)
cam_names=[mujoco.mj_id2name(model,mujoco.mjtObj.mjOBJ_CAMERA,cam_idx) 
                                 for cam_idx in range(model.ncam)]
print("cam_names",cam_names)
camera_id = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_CAMERA, "egocentric")
print("camera_id",camera_id)
cam = mujoco.MjvCamera()
cam.fixedcamid =camera_id
cam.type = mujoco.mjtCamera.mjCAMERA_FIXED
# cam_fov = model.cam_fovy[camera_id]
viewport = mujoco.MjrRect(0,0,resolution[0],resolution[1]) # SVGA?
scn  = mujoco.MjvScene(model, maxgeom=10000)
context  = mujoco.MjrContext(model, mujoco.mjtFontScale.mjFONTSCALE_150.value)

mujoco.mj_step(model, data)
# 更新场景
mujoco.mjv_updateScene(model, data, mujoco.MjvOption(), mujoco.MjvPerturb(), cam, mujoco.mjtCatBit.mjCAT_ALL, scn)
mujoco.mjr_render(viewport, scn,context )
# 提取 RGB 图像
rgb = np.zeros((viewport.height,viewport.width,3),dtype=np.uint8)
depth_raw = np.zeros((viewport.height,viewport.width),dtype=np.float32)
mujoco.mjr_readPixels(rgb,depth_raw,viewport,context )
rgb,depth_raw = np.flipud(rgb),np.flipud(depth_raw)
print("rgb",rgb.shape)
    





D:\cmirobot2.0\data\scene\test.xml
njnt 6
nbody 6
xpos [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
nq 15
qpos [1.  0.  0.  0.  0.  0.  0.  0.  0.3 0.  0.1 1.  0.  0.  0. ]
<class 'numpy.ndarray'>
nv 13
qvel [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
data.ctrl [0. 0.]
data.body(base).xpos [0. 0. 0.]
data.body(base).xmat [0. 0. 0. 0. 0. 0. 0. 0. 0.]
data.joint(base_joint).qpos [1. 0. 0. 0.]
ncam 1
cam_names ['egocentric']
camera_id 0
rgb (600, 800, 3)


In [ ]:
from gymnasium.envs.mujoco.walker2d_v4 import Walker2dEnv
gym.pprint_registry()
gym.spec("Walker2d-v4")

===== classic_control =====
Acrobot-v1             CartPole-v0            CartPole-v1
MountainCar-v0         MountainCarContinuous-v0 Pendulum-v1
===== phys2d =====
phys2d/CartPole-v0     phys2d/CartPole-v1     phys2d/Pendulum-v0
===== box2d =====
BipedalWalker-v3       BipedalWalkerHardcore-v3 CarRacing-v2
LunarLander-v2         LunarLanderContinuous-v2
===== toy_text =====
Blackjack-v1           CliffWalking-v0        FrozenLake-v1
FrozenLake8x8-v1       Taxi-v3
===== tabular =====
tabular/Blackjack-v0   tabular/CliffWalking-v0
===== mujoco =====
Ant-v2                 Ant-v3                 Ant-v4
HalfCheetah-v2         HalfCheetah-v3         HalfCheetah-v4
Hopper-v2              Hopper-v3              Hopper-v4
Humanoid-v2            Humanoid-v3            Humanoid-v4
HumanoidStandup-v2     HumanoidStandup-v4     InvertedDoublePendulum-v2
InvertedDoublePendulum-v4 InvertedPendulum-v2    InvertedPendulum-v4
Pusher-v2              Pusher-v4              Reacher-v2
Reacher-v4         

EnvSpec(id='Walker2d-v4', entry_point='gymnasium.envs.mujoco.walker2d_v4:Walker2dEnv', reward_threshold=None, nondeterministic=False, max_episode_steps=1000, order_enforce=True, autoreset=False, disable_env_checker=False, apply_api_compatibility=False, kwargs={}, namespace=None, name='Walker2d', version=4, additional_wrappers=(), vector_entry_point=None)

In [ ]:
env = gym.make('cmirobot/GridWorld-v0', size=4,render_mode="human")
obs = env.reset()
for _ in range(100):
    action = env.action_space.sample()
    obs, reward, done,truncated , info = env.step(action)
    # env.render()
    if done:
        break

env.close()

In [2]:


env = gym.make("cmirobot/SO100-v0")
obs,info = env.reset()
for _ in range(2):
    action = env.action_space.sample()
    obs, reward, done,truncated , info = env.step(action)
    print(info)
    env.render()
    if done:
        break

env.close()

c:\Users\wangzhian\.conda\envs\nvidia\Lib\site-packages\gymnasium\utils\passive_env_checker.py:135: UserWarning: WARN: The obs returned by the `reset()` method was expecting numpy array dtype to be int64, actual type: uint8
  logger.warn(
c:\Users\wangzhian\.conda\envs\nvidia\Lib\site-packages\gymnasium\utils\passive_env_checker.py:159: UserWarning: WARN: The obs returned by the `reset()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")


ik_err 3.389394372198324
ik_err 3.3091745919944535
ik_err 3.3080519103679
ik_err 3.3040035698002708
ik_err 3.3033942472652056
ik_err 3.300676504615877
ik_err 3.296503077689242
ik_err 3.284695501158559
ik_err 3.2687992584355996
ik_err 3.247445505606931
ik_err 3.219545631235623
ik_err 3.1860914304708006
ik_err 3.1457222937069558
ik_err 3.0951231510145787
ik_err 3.050152717460453
ik_err 3.0113904383281893
ik_err 2.9777852665901388
ik_err 2.951790293192245
ik_err 2.933661722390244
ik_err 2.9211725813525127
ik_err 2.913315771059372
ik_err 2.9095643309278114
ik_err 2.908029849179923
ik_err 2.9043635673506474
ik_err 2.9028671335324203
ik_err 2.899280278483104
ik_err 2.8978194856191246
ik_err 2.8943069365175793
ik_err 2.892879623989455
ik_err 2.889436783124741
ik_err 2.888041002452186
ik_err 2.8846637231924652
ik_err 2.8832977097095958
ik_err 2.879982236746017
ik_err 2.8786443859709476
ik_err 2.87538730537833
ik_err 2.8740761531624237
ik_err 2.8708743503732324
ik_err 2.869588556112818
ik_err 2

c:\Users\wangzhian\.conda\envs\nvidia\Lib\site-packages\gymnasium\utils\passive_env_checker.py:135: UserWarning: WARN: The obs returned by the `step()` method was expecting numpy array dtype to be int64, actual type: uint8
  logger.warn(
c:\Users\wangzhian\.conda\envs\nvidia\Lib\site-packages\gymnasium\utils\passive_env_checker.py:159: UserWarning: WARN: The obs returned by the `step()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")


{'ee_pose': array([-0.00571017, -0.6257471 ,  0.7103584 ,  0.7752633 , -0.8053892 ,
       -1.8427312 ], dtype=float32)}
ik_err 2.45659200996786
ik_err 2.4542470118810575
ik_err 2.45038855058244
ik_err 2.444637453303354
ik_err 2.43697748461714
ik_err 2.4273838310179103
ik_err 2.415816796714665
ik_err 2.402212468375971
ik_err 2.386467103826057
ik_err 2.369621502892034
ik_err 2.3524942318872166
ik_err 2.3353197233965495
ik_err 2.3182580034339004
ik_err 2.301408279900519
ik_err 2.284820812027627
ik_err 2.268506925987577
ik_err 2.252447062271931
ik_err 2.236596838094421
ik_err 2.2208911887640292
ik_err 2.205246682547983
ik_err 2.1895620800045363
ik_err 2.173717137093763
ik_err 2.1575695374372095
ik_err 2.1409496847259315
ik_err 2.1236528895166087
ik_err 2.1054282430991105
ik_err 2.0859631863854884
ik_err 2.064862453297513
ik_err 2.041619619250057
ik_err 2.016412236718657
ik_err 1.9888035377909354
ik_err 1.9577366558394138
ik_err 1.9216076603019663
ik_err 1.8777376317481804
ik_err 1.8210154

In [3]:
from pynput import keyboard

def on_press(key):
    try:
        print(f'按键被按下：{key.char}')
    except AttributeError:
        print(f'特殊按键被按下：{key}')

def on_release(key):
    print(f'按键被释放：{key}')
    if key == keyboard.Key.esc:
        # 停止监听               
        return False
 
# 收集事件直到退出
with keyboard.Listener(on_press=on_press, on_release=on_release) as listener:
    listener.join()

print("继续执行...")


特殊按键被按下：Key.space
按键被释放：Key.space
特殊按键被按下：Key.space
按键被释放：Key.space
特殊按键被按下：Key.space
特殊按键被按下：Key.space
特殊按键被按下：Key.space
特殊按键被按下：Key.space
特殊按键被按下：Key.space
特殊按键被按下：Key.space
特殊按键被按下：Key.space
特殊按键被按下：Key.space
特殊按键被按下：Key.space
特殊按键被按下：Key.space
特殊按键被按下：Key.space
特殊按键被按下：Key.space
特殊按键被按下：Key.space
按键被释放：Key.space
特殊按键被按下：Key.esc
按键被释放：Key.esc
继续执行...


In [1]:
import glfw

title="CMIRobot"
glfw.init()
width, height = glfw.get_video_mode(glfw.get_primary_monitor()).size
window = glfw.create_window(width, height, title, None, None)
glfw.make_context_current(window)
glfw.swap_interval(1)

def _key_callback(self, window, key, scancode, action, mods):
    print(key,action)
    if (key==glfw.KEY_ESCAPE):
        glfw.set_window_should_close(window, True)
 
glfw.set_key_callback(window, _key_callback)

while not glfw.window_should_close(window):
    glfw.swap_buffers(window)
    glfw.poll_events()

# 清理并退出 GLFW
glfw.destroy_window(window)
glfw.terminate()

TypeError: _key_callback() missing 1 required positional argument: 'mods'

In [2]:
import time
from lerobot.teleoperators.keyboard import KeyboardEndEffectorTeleopConfig
config=KeyboardEndEffectorTeleopConfig()
teleop_device=KeyboardEndEffectorTeleop(config=config)
teleop_device.connect()
                        
for _ in range(10):
    action=teleop_device.get_action()
    print(action)   
    time.sleep(1)
    
if teleop_device.is_connected: 
    teleop_device.disconnect()

{'delta_x': 0.0, 'delta_y': 0.0, 'delta_z': 0.0, 'gripper': 1.0}
{'delta_x': 0.0, 'delta_y': 0.0, 'delta_z': 0.0, 'gripper': 1.0}
{'delta_x': 0.0, 'delta_y': 0.0, 'delta_z': 0.0, 'gripper': 1.0}
{'delta_x': 0.0, 'delta_y': 0.0, 'delta_z': 0.0, 'gripper': 1.0}
{'delta_x': 0.0, 'delta_y': 0.0, 'delta_z': 0.0, 'gripper': 1.0}
{'delta_x': 0.0, 'delta_y': 0.0, 'delta_z': 0.0, 'gripper': 1.0}
{'delta_x': 0.0, 'delta_y': 0.0, 'delta_z': 0.0, 'gripper': 1.0}
{'delta_x': 0.0, 'delta_y': 0.0, 'delta_z': 0.0, 'gripper': 1.0}
{'delta_x': 0.0, 'delta_y': 0.0, 'delta_z': 0.0, 'gripper': 1.0}
{'delta_x': 0.0, 'delta_y': 0.0, 'delta_z': 0.0, 'gripper': 1.0}


In [ ]:
from lerobot.teleoperators.keyboard import KeyboardEndEffectorTeleopConfig
config=KeyboardEndEffectorTeleopConfig()
teleop_device=KeyboardEndEffectorTeleop(config=config)
teleop_device.connect()

env = gym.make("cmirobot/SO100-v0")
obs,info = env.reset()
for _ in range(10):
    action=teleop_device.get_action()
    
    action = env.action_space.sample()
    obs, reward, done,truncated , info = env.step(action)
    print(action,obs)
    env.render()
    if done:
        break

env.close()


if teleop_device.is_connected: 
    teleop_device.disconnect()

In [ ]:
data_root = "./data/demo_data"
repo_id="cmirobot"
dataset = LeRobotDataset.create(
                repo_id=repo_id,
                root = data_root, 
                robot_type="omy",
                fps=20, # 20 frames per second
                features={
                    "observation.image": {
                        "dtype": "image",
                        "shape": (256, 256, 3),
                        "names": ["height", "width", "channels"],
                    },
                    "observation.wrist_image": {
                        "dtype": "image",
                        "shape": (256, 256, 3),
                        "names": ["height", "width", "channel"],
                    },
                    "observation.state": {
                        "dtype": "float32",
                        "shape": (6,),
                        "names": ["state"], # x, y, z, roll, pitch, yaw
                    },
                    "action": {
                        "dtype": "float32",
                        "shape": (7,),
                        "names": ["action"], # 6 joint angles and 1 gripper
                    },
                    "obj_init": {
                        "dtype": "float32",
                        "shape": (6,),
                        "names": ["obj_init"], # just the initial position of the object. Not used in training.
                    },
                },
                image_writer_threads=10,
                image_writer_processes=5,
        )

TypeError: MujocoEnv.__init__() missing 1 required positional argument: 'observation_space' was raised from the environment creator for cmirobot/Grab-v0 with kwargs ({'xml_file': '../data/sene/example_scene_y.xml'})

In [ ]:
policy = DiffusionPolicy.from_pretrained("lerobot/diffusion_pusht")


In [ ]:
print("policy.config.input_features",policy.config.input_features)
print("env.observation_space",env.observation_space)
print("policy.config.output_features",policy.config.output_features)
print("env.action_space",env.action_space)

In [ ]:
policy.reset()
numpy_observation, info = env.reset(seed=42)

In [ ]:
device = torch.device("cuda")
rewards = []
frames = []

# Render frame of the initial state
frames.append(env.render())

# Create a directory to store the video of the evaluation
output_directory = Path("outputs/eval/example_pusht_diffusion")
output_directory.mkdir(parents=True, exist_ok=True)

In [ ]:
step = 0
done = False
while not done:
    # Prepare observation for the policy running in Pytorch
    state = torch.from_numpy(numpy_observation["agent_pos"])
    image = torch.from_numpy(numpy_observation["pixels"])

    # Convert to float32 with image from channel first in [0,255]
    # to channel last in [0,1]
    state = state.to(torch.float32)
    image = image.to(torch.float32) / 255
    image = image.permute(2, 0, 1)

    # Send data tensors from CPU to GPU
    state = state.to(device, non_blocking=True)
    image = image.to(device, non_blocking=True)

    # Add extra (empty) batch dimension, required to forward the policy
    state = state.unsqueeze(0)
    image = image.unsqueeze(0)

    # Create the policy input dictionary
    observation = {
        "observation.state": state,
        "observation.image": image,
    }

    # Predict the next action with respect to the current observation
    with torch.inference_mode():
        action = policy.select_action(observation)

    # Prepare the action for the environment
    numpy_action = action.squeeze(0).to("cpu").numpy()

    # Step through the environment and receive a new observation
    numpy_observation, reward, terminated, truncated, info = env.step(numpy_action)
    print(f"{step=} {reward=} {terminated=}")

    # Keep track of all the rewards and frames
    rewards.append(reward)
    frames.append(env.render())

    # The rollout is considered done when the success state is reached (i.e. terminated is True),
    # or the maximum number of iterations is reached (i.e. truncated is True)
    done = terminated | truncated | done
    step += 1

if terminated:
    print("Success!")
else:
    print("Failure!")

# Get the speed of environment (i.e. its number of frames per second).
fps = env.metadata["render_fps"]

# Encode all frames into a mp4 video.
video_path = output_directory / "rollout.mp4"
imageio.mimsave(str(video_path), numpy.stack(frames), fps=fps)

print(f"Video of the evaluation is available in '{video_path}'.")